# Dependency Analysis and Topological Sorting

This notebook demonstrates pollywog's dependency analysis features. You'll learn:
- How dependencies are automatically extracted
- Using `topological_sort()` to resolve calculation order
- Detecting and handling circular dependencies
- Analyzing complex calculation workflows
- Debugging dependency issues

In [ ]:
import pollywog as pw
from pollywog.core import CalcSet, Number, Variable, Category, If
from pollywog.display import display_calcset, set_theme

set_theme("light")

## 1. Automatic Dependency Detection

Pollywog automatically extracts dependencies from expressions by finding all `[variable_name]` references.

In [ ]:
# Create some calculations
calc1 = Number("a", "10")
calc2 = Number("b", "[a] + 5")
calc3 = Number("c", "[b] * 2")
calc4 = Number("d", "[a] + [c]")

# Inspect dependencies
print("Dependencies automatically extracted:\n")
for calc in [calc1, calc2, calc3, calc4]:
    print(f"{calc.name}: {calc.dependencies}")

print("\n💡 Dependencies are extracted from any [variable] in the expression!")

## 2. Topological Sort: Resolving Calculation Order

When calculations depend on each other, they must be executed in the right order. `topological_sort()` figures this out.

In [ ]:
# Create calculations in random order
unsorted = CalcSet([
    Number("final", "[intermediate] * 2"),  # Depends on intermediate
    Number("base", "100"),  # No dependencies
    Number("intermediate", "[base] + [cleanup]"),  # Depends on base and cleanup
    Number("cleanup", "clamp([raw], 0)"),  # Depends on raw
    Variable("raw", "[input]"),  # Depends on external input
])

print("Original order:")
for i, item in enumerate(unsorted.items, 1):
    print(f"{i}. {item.name} (depends on: {item.dependencies})")

# Sort by dependencies
sorted_calcset = unsorted.topological_sort()

print("\nAfter topological_sort():")
for i, item in enumerate(sorted_calcset.items, 1):
    print(f"{i}. {item.name} (depends on: {item.dependencies})")

print("\n✅ Now calculations are in correct execution order!")
print("Each calculation only depends on variables defined earlier.")

## 3. External Dependencies

External dependencies are variables referenced but not defined in the CalcSet (they come from Leapfrog).

In [ ]:
# Workflow that references external Leapfrog variables
workflow = CalcSet([
    Number("Au_clean", "clamp([Au], 0)"),  # Au is external
    Number("Au_scaled", "[Au_clean] * 0.95"),
    Number("tonnage", "[volume] * [density]"),  # volume and density are external
    Number("metal", "[tonnage] * [Au_scaled]"),
])

# Identify external dependencies
defined = {item.name for item in workflow.items}
all_deps = set()
for item in workflow.items:
    all_deps.update(item.dependencies)

external = all_deps - defined

print("Variables defined in this CalcSet:")
print(f"  {sorted(defined)}\n")

print("External dependencies (must exist in Leapfrog):")
print(f"  {sorted(external)}\n")

print("💡 External dependencies must be available in your Leapfrog block model or drillhole database!")

## 4. Circular Dependency Detection

Circular dependencies are impossible to resolve and will cause errors.

In [ ]:
# Create a circular dependency
circular = CalcSet([
    Number("a", "[b] + 1"),  # a depends on b
    Number("b", "[c] + 1"),  # b depends on c
    Number("c", "[a] + 1"),  # c depends on a → CIRCULAR!
])

print("Attempting topological sort on circular dependencies...\n")

try:
    sorted_circular = circular.topological_sort()
    print("Sorted successfully (shouldn't happen!)")
except Exception as e:
    print(f"❌ Error detected: {type(e).__name__}")
    print(f"Message: {e}\n")
    print("✅ Pollywog correctly detected the circular dependency!")
    print("\nThe cycle is: a → b → c → a")

## 5. Analyzing Complex Workflows

Use dependency analysis to understand and debug complex calculation workflows.

In [ ]:
# Create a complex resource estimation workflow
complex_workflow = CalcSet([
    # Preprocessing
    Variable("Au_capped", "clamp([Au], 0, 100)"),
    Variable("Cu_capped", "clamp([Cu], 0, 10)"),
    
    # Domain estimates
    Number("Au_oxide_est", "[Au_oxide_kriged] * [prop_oxide]"),
    Number("Au_sulfide_est", "[Au_sulfide_kriged] * [prop_sulfide]"),
    
    # Composite
    Number("Au_composite", "[Au_oxide_est] + [Au_sulfide_est]"),
    
    # Apply factors
    Number("Au_diluted", "[Au_composite] * 0.95"),
    Number("Au_recovered", "[Au_diluted] * [recovery]"),
    
    # Tonnage
    Number("tonnage", "[volume] * [density]"),
    
    # Metal and value
    Number("Au_ounces", "[tonnage] * [Au_recovered] / 31.1035"),
    Number("value", "[Au_ounces] * [gold_price]"),
    
    # Classification
    Category(
        "class",
        If(
            [
                ("[Au_recovered] < 0.3", "'waste'"),
                ("[Au_recovered] < 1.0", "'low'"),
            ],
            "'high'"
        )
    ),
])

# Analyze dependencies
print("Complex Workflow Analysis")
print("=" * 50)

print(f"\nTotal items: {len(complex_workflow.items)}")

# Count by type
variables = [i for i in complex_workflow.items if i.item_type == 'variable']
numbers = [i for i in complex_workflow.items if i.item_type == 'calculation' and i.calculation_type == 'number']
categories = [i for i in complex_workflow.items if i.item_type == 'calculation' and i.calculation_type == 'category']

print(f"Variables: {len(variables)}")
print(f"Numbers: {len(numbers)}")
print(f"Categories: {len(categories)}")

# Find items with most dependencies
print("\nItems with most dependencies:")
sorted_by_deps = sorted(complex_workflow.items, key=lambda x: len(x.dependencies), reverse=True)
for item in sorted_by_deps[:5]:
    print(f"  {item.name}: {len(item.dependencies)} deps → {item.dependencies}")

# External dependencies
defined = {item.name for item in complex_workflow.items}
all_deps = set()
for item in complex_workflow.items:
    all_deps.update(item.dependencies)
external = all_deps - defined

print(f"\nExternal dependencies ({len(external)}): {sorted(external)}")

# Sort to verify no circular dependencies
print("\nVerifying calculation order...")
sorted_workflow = complex_workflow.topological_sort()
print("✅ No circular dependencies detected!")
print(f"✅ Calculations properly ordered for execution")

## 6. Dependency Visualization

Use pollywog's display features to visualize dependency relationships.

In [ ]:
# Create a workflow with clear dependency chains
dependency_demo = CalcSet([
    # Level 0: No dependencies
    Number("base_value", "100"),
    
    # Level 1: Depend on base_value
    Number("doubled", "[base_value] * 2"),
    Number("squared", "[base_value] * [base_value]"),
    
    # Level 2: Depend on level 1
    Number("combined", "[doubled] + [squared]"),
    
    # Level 3: Final calculation
    Number("final", "[combined] / [base_value]"),
])

display_calcset(dependency_demo)

print("\n📊 The display shows dependency relationships:")
print("   Variables in expressions are clickable/highlighted")
print("   You can see which calculations depend on which variables")

## 7. Debugging Dependency Issues

Common dependency problems and how to find them.

In [ ]:
# Helper function to debug dependencies
def analyze_dependencies(calcset):
    """Comprehensive dependency analysis."""
    print("Dependency Analysis")
    print("=" * 60)
    
    # Get all defined variables
    defined = {item.name for item in calcset.items}
    print(f"\nDefined variables ({len(defined)}):")
    print(f"  {sorted(defined)}")
    
    # Get all dependencies
    all_deps = set()
    for item in calcset.items:
        all_deps.update(item.dependencies)
    
    print(f"\nAll referenced variables ({len(all_deps)}):")
    print(f"  {sorted(all_deps)}")
    
    # Find external dependencies
    external = all_deps - defined
    print(f"\nExternal dependencies ({len(external)}):")
    if external:
        print(f"  {sorted(external)}")
        print("  ⚠️  These must exist in Leapfrog!")
    else:
        print("  None (self-contained workflow)")
    
    # Find unused internal variables
    used_in_deps = set()
    for item in calcset.items:
        used_in_deps.update(dep for dep in item.dependencies if dep in defined)
    
    unused = defined - used_in_deps
    print(f"\nUnused internal variables ({len(unused)}):")
    if unused:
        print(f"  {sorted(unused)}")
        print("  💡 These are probably final outputs (not used by other calcs)")
    else:
        print("  None (all variables are used)")
    
    # Try to sort
    print("\nTopological sort:")
    try:
        sorted_calcset = calcset.topological_sort()
        print("  ✅ Success! No circular dependencies.")
        return sorted_calcset
    except Exception as e:
        print(f"  ❌ Failed: {e}")
        print("  🔍 Look for circular dependencies in the output above")
        return None

# Test with our complex workflow
analyze_dependencies(complex_workflow)

## Summary

**Key concepts:**
- ✅ Dependencies are automatically extracted from `[variable]` references
- ✅ `topological_sort()` orders calculations by dependencies
- ✅ External dependencies must exist in Leapfrog
- ✅ Circular dependencies are detected and reported as errors
- ✅ Use `item.dependencies` to inspect what each calculation needs

**Best practices:**
1. Always use `topological_sort()` before exporting to ensure correct order
2. Check external dependencies match your Leapfrog variable names
3. Use Variables for intermediate steps to keep dependency graphs clean
4. If you get circular dependency errors, trace back through the dependency chain
5. Use the analysis functions above to debug complex workflows

**Common patterns:**
```python
# Check dependencies
calc.dependencies  # Set of variable names

# Sort by dependencies
sorted_calcset = calcset.topological_sort()

# Find external dependencies
defined = {item.name for item in calcset.items}
all_deps = {d for item in calcset.items for d in item.dependencies}
external = all_deps - defined
```